In [1]:
import logging
import os
import sys
from spleeter.separator import Separator
from spleeter.audio.adapter import AudioAdapter
import soundfile as sf
import numpy as np
import librosa
from PIL import Image, ImageOps
from pathlib import Path
import pydub

from scipy.io.wavfile import read

from utils import spleeter_utils

import cv2

from riffusion.spectrogram_image_converter import SpectrogramImageConverter
from riffusion.spectrogram_converter import SpectrogramConverter
from riffusion.spectrogram_params import SpectrogramParams

#sample_audio_path = "pop.00027_seg4_full.wav"

/Users/zachary/miniconda3/envs/mel-gen/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [81]:
img_converter = SpectrogramImageConverter(SpectrogramParams(sample_rate=44100, min_frequency=0, max_frequency=10000))

In [82]:
audio_loader = AudioAdapter.default()
waveform, _ = audio_loader.load(sample_audio_path, sample_rate=44100)
# print(waveform, type(waveform))
print(np.shape(np.int64(waveform / (np.max(np.abs(waveform))) * (2**15-1))))
print(np.shape(waveform))
byte_count = (np.int8(waveform / (np.max(np.abs(waveform))) * (2**15-1))).copy().tobytes()
print(len(byte_count))
print(waveform.dtype.itemsize)
waveform =  np.int16(waveform / (np.max(np.abs(waveform))) * (2**15-1))
song = pydub.AudioSegment(
    waveform.tobytes(),
    frame_rate=44100,
    sample_width=waveform.dtype.itemsize, 
    channels=1
)

# song = pydub.AudioSegment.from_file(sample_audio_path, sr=44100)

out_img = img_converter.spectrogram_image_from_audio(song)
out_img.save("temp_spec.jpg", exif=out_img.getexif())

(225351, 1)
(225351, 1)
225351
4


In [4]:
recon_filepath = "train-150l200h/source/pop.00027_seg4.jpg"

recon_img =  Image.open(recon_filepath).convert("RGB")

img_converter_to_audio = SpectrogramImageConverter(SpectrogramParams(sample_rate=44100, min_frequency=0, max_frequency=10000))

out_audio_recon = img_converter_to_audio.audio_from_spectrogram_image(recon_img, apply_filters=True).set_channels(2)
out_audio_recon.export("temp_wav_good_edges.wav", format="wav")

<_io.BufferedRandom name='temp_wav_good_edges.wav'>

In [1]:
# from utils import spleeter_utils
# sample_audio_path = "raw-audio/blues.00000.wav"
# info = spleeter_utils.separate_audio(sample_audio_path, fs=44100)

In [ ]:
# write_wav_file(accompaniment_audio, os.path.join(full_clip_dir, f'{audio_filename}_seg_test_bgnd.wav'), fs=fs,  verbose=verbose)
# write_wav_file(vocal_audio, os.path.join(full_clip_dir, f'{audio_filename}_seg_test_voc.wav'), fs=fs,  verbose=verbose)
# write_wav_file(full_audio, os.path.join(full_clip_dir, f'{audio_filename}_seg_test_full.wav'), fs=fs,  verbose=verbose)